### Data analysis workflow
Normalization -> Highly variable genes (HVGs) -> Differential expression genes (DEG) analysis -> Merge files into a total file.

### Normalization and HVGs
Norm_hvg.py

* Unable to run because no file here, only for displaying.

In [ ]:
import argparse
import os
from pathlib import Path
from typing import Dict, List, Tuple
import numpy as np
import pandas as pd
import scanpy as sc

os.environ.setdefault("NUMBA_CACHE_DIR", "./numba_cache")


def resolve_file(candidates: List[str], base_dir: Path) -> Path:
    """Return the first existing path from candidates (absolute or relative to base_dir/..)."""
    for name in candidates:
        path = Path(name)
        if path.is_file():
            return path
        for root in (base_dir, base_dir.parent):
            maybe = root / name
            if maybe.is_file():
                return maybe
    raise FileNotFoundError(f"None of these files exist: {candidates}")


def load_metadata(meta_candidates: List[str], base_dir: Path) -> pd.DataFrame:
    meta_path = resolve_file(meta_candidates, base_dir)
    with open(meta_path, "rb") as fh:
        header = fh.read(4)

    try:
        if header.startswith(b"PK") or meta_path.suffix.lower() in {".xls", ".xlsx"}:
            meta = pd.read_excel(meta_path)
        else:
            meta = pd.read_csv(
                meta_path,
                encoding="utf-8",
                encoding_errors="replace",
                engine="python",
                on_bad_lines="skip",
            )
    except Exception:
        meta = pd.read_excel(meta_path)

    if "Barcode" not in meta.columns:
        raise KeyError(f"{meta_path} is missing required column 'Barcode'")
    meta = meta.set_index("Barcode")
    return meta


def stratified_sample(meta: pd.DataFrame, celltype_col: str, target_total: int, seed: int) -> Tuple[List[str], Dict[str, int], Dict[str, int]]:
    """Sample cells per cell type to approach target_total."""
    counts = meta[celltype_col].value_counts()
    available = counts.to_dict()
    total = counts.sum()

    if total <= target_total:
        selected = meta.index.to_list()
        sampled = {ct: counts[ct] for ct in counts.index}
        return selected, available, sampled

    fractions = counts / total
    ideal = fractions * target_total
    sample_sizes = ideal.apply(np.floor).astype(int)
    remainder = target_total - sample_sizes.sum()

    fractional_part = (ideal - sample_sizes).sort_values(ascending=False)
    for ct in fractional_part.index:
        if remainder <= 0:
            break
        room = counts[ct] - sample_sizes[ct]
        add = min(room, remainder)
        sample_sizes[ct] += add
        remainder -= add

    if remainder > 0:
        for ct in counts.index:
            if remainder == 0:
                break
            room = counts[ct] - sample_sizes[ct]
            if room <= 0:
                continue
            add = min(room, remainder)
            sample_sizes[ct] += add
            remainder -= add

    rng = np.random.default_rng(seed)
    selected: List[str] = []
    for ct, n in sample_sizes.items():
        pool = meta[meta[celltype_col] == ct].index.to_numpy()
        if n <= 0:
            continue
        if n > len(pool):
            n = len(pool)
        chosen = rng.choice(pool, size=n, replace=False)
        selected.extend(chosen.tolist())

    sampled = sample_sizes.to_dict()
    return selected, available, sampled


def main():
    parser = argparse.ArgumentParser(description="Subsample cells (~10k) by cell type, normalize, and compute HVGs.")
    parser.add_argument("--target_total", type=int, default=10000, help="Approximate total cells after sampling (default: 10000)")
    parser.add_argument("--seed", type=int, default=42, help="Random seed for reproducible sampling")
    parser.add_argument("--celltype_col", type=str, default="Cell_type", help="Column in metadata that stores cell type labels")
    args = parser.parse_args()

    base_dir = Path(__file__).resolve().parent
    h5_path = resolve_file(["GSE174367_snRNA-seq_filtered_feature_bc_matrix.h5"], base_dir)
    meta = load_metadata(["GSE174367_snRNA-seq_cell_meta.csv", "PFC_metadata_whole.csv"], base_dir)

    if args.celltype_col not in meta.columns:
        raise KeyError(f"Metadata missing column '{args.celltype_col}'. Available columns: {list(meta.columns)}")
    meta = meta[meta[args.celltype_col].notna()].copy()

    raw_adata = sc.read_10x_h5(h5_path)
    raw_adata.var_names_make_unique()
    raw_adata = raw_adata[raw_adata.obs_names.isin(meta.index)].copy()
    meta_aligned = meta.loc[raw_adata.obs_names]
    raw_adata.obs = raw_adata.obs.join(meta_aligned, how="left")

    selected, available_counts, sampled_counts = stratified_sample(
        raw_adata.obs, args.celltype_col, args.target_total, args.seed
    )
    adata = raw_adata[selected, :].copy()

    before_sum = sum(available_counts.values())
    after_sum = len(selected)
    print(f"Sampling {after_sum}/{before_sum} cells (~{args.target_total} target).")

    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)

    batch_key = "Batch" if "Batch" in adata.obs.columns else None
    sc.pp.highly_variable_genes(
        adata,
        n_top_genes=2000,
        flavor="seurat_v3",
        batch_key=batch_key,
    )

    hvg_df_raw = adata.var[adata.var["highly_variable"]].copy()
    hvg_df = hvg_df_raw.assign(
        gene_name=hvg_df_raw.index,
        mean=hvg_df_raw["means"],
        variance=hvg_df_raw["variances"],
        variance_norm=hvg_df_raw["variances_norm"],
        dispersion=hvg_df_raw["variances_norm"],
    )[["gene_name", "mean", "variance", "variance_norm", "dispersion"]]

    out_raw = base_dir / "sampled_raw_counts.h5ad"
    out_norm = base_dir / "sampled_normalized_expression.h5ad"
    out_hvg = base_dir / "sampled_norm_hvg.h5ad"
    out_hvg_csv = base_dir / "sampled_HVG.csv"
    out_counts = base_dir / "sampled_celltype_counts.csv"
    out_meta = base_dir / "GSE174367_snRNA-seq_cell_meta_sampled.csv"

    raw_subset = raw_adata[selected, :].copy()
    raw_subset.write(out_raw, compression="gzip")
    adata.write(out_norm, compression="gzip")
    adata[:, adata.var["highly_variable"]].write(out_hvg, compression="gzip")
    hvg_df.to_csv(out_hvg_csv, index=False)

    # Save sampled metadata with Barcode column preserved
    meta_sampled = adata.obs.copy()
    meta_sampled.insert(0, "Barcode", meta_sampled.index)
    meta_sampled.to_csv(out_meta, index=False)

    counts_df = pd.DataFrame(
        {
            "cell_type": list(available_counts.keys()),
            "available": list(available_counts.values()),
            "sampled": [sampled_counts.get(ct, 0) for ct in available_counts.keys()],
        }
    )
    counts_df.to_csv(out_counts, index=False)

    print(f"HVGs found: {hvg_df.shape[0]}")
    print(f"Saved: {out_raw.name}, {out_norm.name}, {out_hvg.name}, {out_hvg_csv.name}, {out_counts.name}, {out_meta.name}")


if __name__ == "__main__":
    main()


### DEGs and merge files into total file
Deg_and_merge.py

* Unable to run because no file, only for displaying.

In [ ]:
import argparse
import os
import re
from pathlib import Path
from typing import Dict, List, Tuple
import pandas as pd
import scanpy as sc

os.environ.setdefault("NUMBA_CACHE_DIR", "./numba_cache")


def resolve_file(candidates: List[str], base_dir: Path) -> Path:
    for name in candidates:
        path = Path(name)
        if path.is_file():
            return path
        for root in (base_dir, base_dir.parent):
            maybe = root / name
            if maybe.is_file():
                return maybe
    raise FileNotFoundError(f"None of these files exist: {candidates}")


def load_gene_list(base_dir: Path) -> List[str]:
    gene_path = resolve_file(["master_dictionary_gene_unique.csv"], base_dir)
    gene_dict = pd.read_csv(
        gene_path,
        encoding="utf-8",
        encoding_errors="replace",
        engine="python",
        on_bad_lines="skip",
    )
    gene_list = (
        gene_dict["gene_symbol"]
        .dropna()
        .astype(str)
        .str.strip()
        .unique()
        .tolist()
    )
    return gene_list


def choose_reference_and_target(diag_values: List[str]) -> Tuple[str, str]:
    diag_set = set(diag_values)
    if not diag_set:
        raise ValueError("Diagnosis column is empty after filtering.")

    if "Control" in diag_set:
        ref = "Control"
    elif "CT" in diag_set:
        ref = "CT"
    else:
        ref = sorted(diag_set)[0]

    non_ref = [g for g in diag_values if g != ref]
    if not non_ref:
        raise ValueError(f"Only one diagnosis group ({ref}) present; cannot run DEG.")

    target = "AD" if "AD" in non_ref else non_ref[0]
    return ref, target


def main():
    parser = argparse.ArgumentParser(description="DEG on sampled dataset + merged outputs.")
    parser.add_argument("--adata", type=str, default="sampled_normalized_expression.h5ad", help="Input AnnData file")
    parser.add_argument("--celltype_col", type=str, default="Cell_type", help="Cell type column in obs")
    parser.add_argument("--diagnosis_col", type=str, default="Diagnosis", help="Diagnosis column in obs")
    parser.add_argument("--min_cells", type=int, default=20, help="Minimum cells per cell type to run DEG")
    args = parser.parse_args()

    base_dir = Path(__file__).resolve().parent
    adata_path = resolve_file([args.adata, "normalized_expression.h5ad"], base_dir)

    adata = sc.read(adata_path)
    adata.var_names_make_unique()

    if args.celltype_col not in adata.obs.columns and "Cell.Type" in adata.obs.columns:
        args.celltype_col = "Cell.Type"

    if args.celltype_col not in adata.obs.columns:
        raise KeyError(f"Missing cell type column: {args.celltype_col}")
    if args.diagnosis_col not in adata.obs.columns:
        raise KeyError(f"Missing diagnosis column: {args.diagnosis_col}")

    gene_list = load_gene_list(base_dir)
    genes_in_data = [g for g in gene_list if g in adata.var_names]
    if not genes_in_data:
        raise ValueError("No genes from master_dictionary_gene_unique.csv found in expression matrix.")
    adata = adata[:, genes_in_data].copy()
    print(f"Using {len(genes_in_data)} genes from master_dictionary_gene_unique.csv")

    cell_types = sorted(adata.obs[args.celltype_col].dropna().unique())
    out_dir = base_dir
    logfc_dict: Dict[str, pd.Series] = {}
    deg_files: List[Tuple[str, Path]] = []

    for ct in cell_types:
        ad_ct = adata[adata.obs[args.celltype_col] == ct].copy()
        if ad_ct.n_obs < args.min_cells:
            print(f"Skip {ct}: too few cells ({ad_ct.n_obs})")
            continue

        diag_values = ad_ct.obs[args.diagnosis_col].dropna().unique().tolist()
        if len(diag_values) < 2:
            print(f"Skip {ct}: only one diagnosis group ({diag_values})")
            continue

        ref, target = choose_reference_and_target(diag_values)
        sc.tl.rank_genes_groups(
            ad_ct,
            groupby=args.diagnosis_col,
            reference=ref,
            method="wilcoxon",
        )
        sc.tl.filter_rank_genes_groups(
            ad_ct,
            min_in_group_fraction=0.1,
            max_out_group_fraction=0.1,
            min_fold_change=1.5,
        )

        deg_df = sc.get.rank_genes_groups_df(ad_ct, group=None)
        safe_ct = re.sub(r"[^0-9A-Za-z._-]+", "_", ct)
        out_path = out_dir / f"Sampled_DEG_{safe_ct}.csv"
        deg_df.to_csv(out_path, index=False)
        deg_files.append((ct, out_path))

        deg_target = deg_df[deg_df["group"] == target] if "group" in deg_df.columns else deg_df
        logfc_dict[ct] = deg_target.set_index("names")["logfoldchanges"]
        print(f"{ct}: {ad_ct.n_obs} cells, DEG -> {out_path.name}")

    if not deg_files:
        raise RuntimeError("No DEG files were generated. Check filters and metadata.")

    logfc_df = pd.DataFrame(logfc_dict)
    logfc_df.insert(0, "Gene", logfc_df.index)
    out_logfc = out_dir / "Sampled_DEG_Log_CellType.csv"
    logfc_df.to_csv(out_logfc, index=False)

    merged_list = []
    drop_cols = {"Gene", "ASC", "EX", "INH", "MG", "ODC", "CellType", "OPC", "PER.END", "cell_type"}
    for ct, fp in deg_files:
        df = pd.read_csv(fp)
        df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")
        df.insert(0, "cell_type", ct)
        if df.shape[1] > 7:
            df = df.iloc[:, :7]
        merged_list.append(df)

    merged_df = pd.concat(merged_list, ignore_index=True)
    out_merged = out_dir / "Sampled_DEG_celltype_merged.csv"
    merged_df.to_csv(out_merged, index=False)

    print(f"LogFC matrix: {out_logfc.name}")
    print(f"Merged DEG: {out_merged.name} (from {len(merged_list)} cell types)")


if __name__ == "__main__":
    main()
